## 数据爬取并保存

In [23]:
import requests
from pprint import pprint
books=[]
def add_book(table):
    for book in table:
        books.append({
            '小说名':book.get('title'),
            '作者名':book.get('author'),
            '作者主页':book.get('author_url'),
            '热度值':book.get('number'),
            '热度单位':book.get('unit'),
            '字数':book.get('words_num'),
            '类别1':book.get('category1_name'),
            '类别2':book.get('category2_name'),
            '最新章节':book.get('latest_chapter_title'),
            '更新时间':book.get('update_time'),
            '小说链接':book.get('book_url'),
        })
for i in range(5):
    url=f'https://www.qimao.com/api/rank/book-list?is_girl=0&rank_type=1&date_type=1&date=202304&page={i+1}'
    r=requests.get(url,timeout=10).json()
    add_book(r.get('data').get('table_data'))

In [24]:
len(books)

100

In [25]:
import pandas as pd
df=pd.DataFrame(books)
df.to_csv('爬取数据.csv')

## 数据处理（热度补全，类别整合，删除原列）

In [26]:
df_处理=df
df_处理

,小说名,作者名,作者主页,热度值,热度单位,字数,类别1,类别2,最新章节,更新时间,小说链接
0,盖世神医,狐颜乱语,https://www.qimao.com/zuozhe/%E7%8B%90%E9%A2%9...,385.1,万,391.36万字,都市人生,都市修仙,第1908章 黄金圣树果实,2023-05-31 22:43:15,https://www.qimao.com/shuku/195958/
1,绝世强龙,张龙虎,https://www.qimao.com/zuozhe/%E5%BC%A0%E9%BE%9...,211.2,万,415.84万字,都市人生,都市高手,第1924章 交互,2023-05-31 19:52:16,https://www.qimao.com/shuku/215780/
2,寒门枭士,北川,https://www.qimao.com/zuozhe/%E5%8C%97%E5%B7%9...,123.2,万,321.01万字,历史,架空历史,第1549章 要点脸,2023-05-31 22:41:20,https://www.qimao.com/shuku/215169/
3,武逆九千界,虚尘,https://www.qimao.com/zuozhe/%E8%99%9A%E5%B0%9...,63.2,万,712.99万字,玄幻奇幻,东方玄幻,第3316章 大帝十三斩，碾压真灵九变,2023-05-31 21:10:35,https://www.qimao.com/shuku/215086/
4,混沌剑帝,运也,https://www.qimao.com/zuozhe/%E8%BF%90%E4%B9%9...,56.7,万,331.92万字,玄幻奇幻,东方玄幻,第1487章 敢嚣张跋扈？,2023-05-31 00:40:12,https://www.qimao.com/shuku/213189/
...,...,...,...,...,...,...,...,...,...,...,...
95,控卫凶猛,流浪沙鸥,https://www.qimao.com/zuozhe/%E6%B5%81%E6%B5%A...,8.4,万,229.77万字,体育,篮球风云,第598章 无非是煎蛋还是水煮蛋的区别,2023-05-31 17:12:23,https://www.qimao.com/shuku/219883/
96,山野诡闻笔记,吴大胆,https://www.qimao.com/zuozhe/%E5%90%B4%E5%A4%A...,8.3,万,184.72万字,奇闻异事,奇门秘术,第720章 立处生虫,2023-05-31 00:59:06,https://www.qimao.com/shuku/214783/
97,不败战神,必焱,https://www.qimao.com/zuozhe/%E5%BF%85%E7%84%B...,8.0,万,395.97万字,都市人生,都市高手,第1970章 那片星空！,2022-11-22 17:51:16,https://www.qimao.com/shuku/149774/
98,重回1991,南三石,https://www.qimao.com/zuozhe/%E5%8D%97%E4%B8%8...,8.0,万,502.39万字,都市人生,商战职场,第2474章 各种搞事情,2023-05-31 07:21:18,https://www.qimao.com/shuku/209888/


In [31]:
def compute_热度(row):
    return row['热度值']+row['热度单位']
df_处理['热度']=df_处理.apply(compute_热度,axis=1)

In [32]:
def compute_类别(row):
    return row['类别1']+'/'+row['类别2']
df_处理['类别']=df_处理.apply(compute_类别,axis=1)

In [35]:
df_处理.drop(['热度值','热度单位','类别1','类别2'],axis=1,inplace=True)

In [36]:
df_处理

,小说名,作者名,作者主页,字数,最新章节,更新时间,小说链接,热度,类别
0,盖世神医,狐颜乱语,https://www.qimao.com/zuozhe/%E7%8B%90%E9%A2%9...,391.36万字,第1908章 黄金圣树果实,2023-05-31 22:43:15,https://www.qimao.com/shuku/195958/,385.1万,都市人生/都市修仙
1,绝世强龙,张龙虎,https://www.qimao.com/zuozhe/%E5%BC%A0%E9%BE%9...,415.84万字,第1924章 交互,2023-05-31 19:52:16,https://www.qimao.com/shuku/215780/,211.2万,都市人生/都市高手
2,寒门枭士,北川,https://www.qimao.com/zuozhe/%E5%8C%97%E5%B7%9...,321.01万字,第1549章 要点脸,2023-05-31 22:41:20,https://www.qimao.com/shuku/215169/,123.2万,历史/架空历史
3,武逆九千界,虚尘,https://www.qimao.com/zuozhe/%E8%99%9A%E5%B0%9...,712.99万字,第3316章 大帝十三斩，碾压真灵九变,2023-05-31 21:10:35,https://www.qimao.com/shuku/215086/,63.2万,玄幻奇幻/东方玄幻
4,混沌剑帝,运也,https://www.qimao.com/zuozhe/%E8%BF%90%E4%B9%9...,331.92万字,第1487章 敢嚣张跋扈？,2023-05-31 00:40:12,https://www.qimao.com/shuku/213189/,56.7万,玄幻奇幻/东方玄幻
...,...,...,...,...,...,...,...,...,...
95,控卫凶猛,流浪沙鸥,https://www.qimao.com/zuozhe/%E6%B5%81%E6%B5%A...,229.77万字,第598章 无非是煎蛋还是水煮蛋的区别,2023-05-31 17:12:23,https://www.qimao.com/shuku/219883/,8.4万,体育/篮球风云
96,山野诡闻笔记,吴大胆,https://www.qimao.com/zuozhe/%E5%90%B4%E5%A4%A...,184.72万字,第720章 立处生虫,2023-05-31 00:59:06,https://www.qimao.com/shuku/214783/,8.3万,奇闻异事/奇门秘术
97,不败战神,必焱,https://www.qimao.com/zuozhe/%E5%BF%85%E7%84%B...,395.97万字,第1970章 那片星空！,2022-11-22 17:51:16,https://www.qimao.com/shuku/149774/,8.0万,都市人生/都市高手
98,重回1991,南三石,https://www.qimao.com/zuozhe/%E5%8D%97%E4%B8%8...,502.39万字,第2474章 各种搞事情,2023-05-31 07:21:18,https://www.qimao.com/shuku/209888/,8.0万,都市人生/商战职场


In [37]:
df_处理.to_csv('爬取数据-处理.csv')

## 数据清洗（删除字数少于500万字的）

TypeError: cannot convert the series to <class 'float'>

In [49]:
df_清洗=df_处理
# df_清洗 = df_清洗.drop(df_清洗[(float(df_清洗['字数'].replace('万字','')) < 500)].index)
df_清洗 = df_清洗.drop(df_清洗[df_清洗['字数'].str.replace('万字','').astype(float) < 500].index)
# df_清洗 = df_清洗.drop(df_清洗[float(df_清洗['字数'].str.replace('万字','')) < 500].index)
df_清洗.index = range(len(df_清洗))
df_清洗.to_csv('爬取数据_处理.csv')
